<center><img src="https://www.inteli.edu.br/wp-content/uploads/2021/08/20172028/marca_1-2.png" width="50%" height="50%"/></center>

<h1 align='center'><b>IA para Marketing: Monitoramento de campanhas utilizando processamento de linguagem natural (PLN)<b></h1>

<center><img src="https://upload.wikimedia.org/wikipedia/commons/c/c2/Btg-logo-blue.svg" width="50%" height="50%"/></center>

<h2 align='center'>O Banco BTG Pactual enfrenta um desafio na área de Marketing em entender as necessidades e demandas dos clientes de maneira fácil e rápida nas redes sociais. A solução proposta para esse problema foi o desenvolvimento de uma Inteligência Artificial utilizando processamento de linguagem natural (PLN), capaz de monitorar as campanhas de marketing, voltadas para o Instagram. O objetivo principal dessa solução é rastrear os dados em tempo real, analisar e interpretar as mensagens e comentários enviados pelos clientes na rede social, a fim de identificar as necessidades e demandas de forma precisa e eficiente.</h2>

---

# Sobre os dados

Esse projeto estará utilizando dados coletados e tratados pela equipe de Automation do BTG Pactual, o qual disponibilizou o dataset. Com base nas informações dispostas nesse dataset, realizaremos insights a cerca dos comentários feitos nos posts do Instagram do próprio banco. Vale lembrar que os dados estão anonimizados e resguardados para manter a privacidade e ética com os usuários e com o banco.

# Preparação do ambiente

In [188]:
!pip install nltk


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\vitor\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [189]:
!pip install openpyxl


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\vitor\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [190]:
!pip install plotly.express


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\vitor\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [191]:
# import re
# import plotly.express as px
# import matplotlib as plt
import pandas as pd

import nltk
nltk.download('punkt')
from nltk.stem.snowball import SnowballStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vitor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vitor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vitor\AppData\Roaming\nltk_data...


In [192]:
import re
import plotly.express as px
# import matplotlib as plt

In [193]:
df = pd.read_csv('../Base/2-base_10052023.xlsx - Sheet1.csv')
df.head()

,id,"""dataPublicada""","""autor""","""texto""","""sentimento""","""tipoInteracao""","""anomalia""","""probabilidadeAnomalia""","""linkPost""","""processado""","""contemHyperlink"""
0,1,"""2022-03-04 09:38:00""",v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,marcação,1,52,https://www.instagram.com/p/CarqO6drRux/,0,0
1,2,"""2022-03-04 09:29:00""",winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,marcação,0,43,https://www.instagram.com/p/CarpQUluh7z/,0,0
2,3,"""2022-03-04 09:20:00""",marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,marcação,0,45,https://www.instagram.com/p/CaroK9Nu8tC/,0,0
3,4,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0
4,5,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0


In [194]:
df = df.rename(columns={'"anomalia"' : 'anomalia', '"dataPublicada"' : 'dataPublicada', '"autor"' : 'autor', '"texto"' : 'texto', '"sentimento"' : 'sentimento', '"tipoInteracao"' : 'tipoInteracao', '"probabilidadeAnomalia"' : 'probabilidadeAnomalia', '"linkPost"' : 'linkPost', '"processado"' : 'processado',  '"contemHyperlink"' : 'contemHyperlink' })

# 1.0 Análise exploratória

In [195]:
sentimento_counts = df['sentimento'].value_counts()

In [196]:
fig = px.bar(sentimento_counts, x=sentimento_counts.index, y='sentimento',
             labels={'sentimento': 'Contagem', 'x': 'sentimento'},
             title='sentimentos distribuídos')
fig.show()

In [197]:
df['autor'].nunique()

5839

In [198]:
autores = df['autor'].value_counts().sort_values(ascending=False)
autores

amgcapitalinvest           237
btgpactual                 162
luizantoniojovinodossan     87
clubefiinews                75
marcelo.flora               68
                          ... 
fiafinance                   1
annatellesoficial            1
dumarcilio06                 1
henryroges                   1
bfmarilia                    1
Name: autor, Length: 5839, dtype: int64

In [199]:
fig = px.bar(autores, x=autores.index, y=autores.values,
             labels={'x': 'autor', 'y': 'contagem'},
             title='autores com mais comentários')
fig.update_traces(width=2)
fig.show()

# 1.0 Processamento dos dados

In [200]:
df = df.loc[df['autor'] != 'btgpactual'] #Remoção de replies do BTG

In [201]:
df.head()

,id,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink
0,1,"""2022-03-04 09:38:00""",v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,marcação,1,52,https://www.instagram.com/p/CarqO6drRux/,0,0
1,2,"""2022-03-04 09:29:00""",winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,marcação,0,43,https://www.instagram.com/p/CarpQUluh7z/,0,0
2,3,"""2022-03-04 09:20:00""",marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,marcação,0,45,https://www.instagram.com/p/CaroK9Nu8tC/,0,0
3,4,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0
4,5,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0


## 1.1.0 Pré-processamento de texto

### 1.2.0 Definição de funções

### 1.2.1 Tokenização

In [202]:
def tokenizer(comment):
    if isinstance(comment, str): # verifica se é um comentário do tipo string
        tokens = nltk.word_tokenize(comment)
        return tokens
    else:
        return []

In [203]:
df['tokenized'] = df['texto'].apply(tokenizer)
df

,id,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink,tokenized
0,1,"""2022-03-04 09:38:00""",v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,marcação,1,52,https://www.instagram.com/p/CarqO6drRux/,0,0,"[Confira, os, resultados, dos, nossos, fundos,..."
1,2,"""2022-03-04 09:29:00""",winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,marcação,0,43,https://www.instagram.com/p/CarpQUluh7z/,0,0,"[A, Alvarez, &, Marsal, estará, conosco, no, S..."
2,3,"""2022-03-04 09:20:00""",marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,marcação,0,45,https://www.instagram.com/p/CaroK9Nu8tC/,0,0,"[#, Repost, btgpactual, with, make_repost, ・・・..."
3,4,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0,"[Minuto, touro, de, ouro]"
4,5,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0,"[@, ricktolledo, Sim]"
...,...,...,...,...,...,...,...,...,...,...,...,...
12350,3044013,"""2022-11-30 11:51:00""",onixcapital,Uma noite de encontros e muito conhecimento on...,NEUTRAL,marcação,1,54,https://www.instagram.com/reel/CllsqI7goSC/,0,0,"[Uma, noite, de, encontros, e, muito, conhecim..."
12351,3044212,"""2022-11-30 12:25:00""",amgcapitalinvest,Erros financeiros para eliminar antes dos 30 a...,NEUTRAL,marcação,0,39,https://www.instagram.com/p/Cllw0CmNGkX/,0,0,"[Erros, financeiros, para, eliminar, antes, do..."
12352,3044213,"""2022-11-30 12:18:00""",jayipacderota,Estou muito grato por todo o esforço que você ...,POSITIVE,comentário,1,52,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0,"[Estou, muito, grato, por, todo, o, esforço, q..."
12353,3044412,"""2022-11-30 12:50:00""",moinho_cultural,"""👉GUADAKAN está chegando!!!! A aula pública e...",NEUTRAL,marcação,0,43,https://www.instagram.com/reel/ClluX-cAoaf/,0,0,"[``, 👉GUADAKAN, está, chegando, !, !, !, !, A,..."


In [204]:
df['tokenized'].head()

0    [Confira, os, resultados, dos, nossos, fundos,...
1    [A, Alvarez, &, Marsal, estará, conosco, no, S...
2    [#, Repost, btgpactual, with, make_repost, ・・・...
3                            [Minuto, touro, de, ouro]
4                                [@, ricktolledo, Sim]
Name: tokenized, dtype: object

### 1.2.2 Remoção de stopwords


In [205]:
stop_words = set(stopwords.words('portuguese'))

In [206]:
def remove_stopwords(tokens):
    if isinstance(tokens, list): # verifica se é um comentário do tipo string
        comments_filtered = []
        for token in tokens:
            if token.lower() not in stop_words:
                comments_filtered.append(token)
        return comments_filtered
    else:
        return []

In [207]:
df['no_stopwords'] = df['tokenized'].apply(remove_stopwords)
df

,id,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink,tokenized,no_stopwords
0,1,"""2022-03-04 09:38:00""",v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,marcação,1,52,https://www.instagram.com/p/CarqO6drRux/,0,0,"[Confira, os, resultados, dos, nossos, fundos,...","[Confira, resultados, fundos, mês, fevereiro/2..."
1,2,"""2022-03-04 09:29:00""",winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,marcação,0,43,https://www.instagram.com/p/CarpQUluh7z/,0,0,"[A, Alvarez, &, Marsal, estará, conosco, no, S...","[Alvarez, &, Marsal, estará, conosco, Sportain..."
2,3,"""2022-03-04 09:20:00""",marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,marcação,0,45,https://www.instagram.com/p/CaroK9Nu8tC/,0,0,"[#, Repost, btgpactual, with, make_repost, ・・・...","[#, Repost, btgpactual, with, make_repost, ・・・..."
3,4,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0,"[Minuto, touro, de, ouro]","[Minuto, touro, ouro]"
4,5,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0,"[@, ricktolledo, Sim]","[@, ricktolledo, Sim]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12350,3044013,"""2022-11-30 11:51:00""",onixcapital,Uma noite de encontros e muito conhecimento on...,NEUTRAL,marcação,1,54,https://www.instagram.com/reel/CllsqI7goSC/,0,0,"[Uma, noite, de, encontros, e, muito, conhecim...","[noite, encontros, conhecimento, onde, cliente..."
12351,3044212,"""2022-11-30 12:25:00""",amgcapitalinvest,Erros financeiros para eliminar antes dos 30 a...,NEUTRAL,marcação,0,39,https://www.instagram.com/p/Cllw0CmNGkX/,0,0,"[Erros, financeiros, para, eliminar, antes, do...","[Erros, financeiros, eliminar, antes, 30, anos..."
12352,3044213,"""2022-11-30 12:18:00""",jayipacderota,Estou muito grato por todo o esforço que você ...,POSITIVE,comentário,1,52,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0,"[Estou, muito, grato, por, todo, o, esforço, q...","[grato, todo, esforço, rendeu, investimento, R..."
12353,3044412,"""2022-11-30 12:50:00""",moinho_cultural,"""👉GUADAKAN está chegando!!!! A aula pública e...",NEUTRAL,marcação,0,43,https://www.instagram.com/reel/ClluX-cAoaf/,0,0,"[``, 👉GUADAKAN, está, chegando, !, !, !, !, A,...","[``, 👉GUADAKAN, chegando, !, !, !, !, aula, pú..."


### 1.2.3 Stemming

In [208]:
sn_stemmer = SnowballStemmer('portuguese')

In [209]:
def stemming(tokens):
    if isinstance(tokens, list): # verifica se é um comentário do tipo string
        stem_list = []
        for token in tokens: 
            stem_list.append(sn_stemmer.stem(token))
        return stem_list
    else:
        return []

In [210]:
df['stemmed'] = df['no_stopwords'].apply(stemming)
df

,id,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink,tokenized,no_stopwords,stemmed
0,1,"""2022-03-04 09:38:00""",v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,marcação,1,52,https://www.instagram.com/p/CarqO6drRux/,0,0,"[Confira, os, resultados, dos, nossos, fundos,...","[Confira, resultados, fundos, mês, fevereiro/2...","[conf, result, fund, mês, fevereiro/22, ., tod..."
1,2,"""2022-03-04 09:29:00""",winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,marcação,0,43,https://www.instagram.com/p/CarpQUluh7z/,0,0,"[A, Alvarez, &, Marsal, estará, conosco, no, S...","[Alvarez, &, Marsal, estará, conosco, Sportain...","[alvarez, &, marsal, estar, conosc, sportainme..."
2,3,"""2022-03-04 09:20:00""",marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,marcação,0,45,https://www.instagram.com/p/CaroK9Nu8tC/,0,0,"[#, Repost, btgpactual, with, make_repost, ・・・...","[#, Repost, btgpactual, with, make_repost, ・・・...","[#, repost, btgpactual, with, make_repost, ・・・..."
3,4,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0,"[Minuto, touro, de, ouro]","[Minuto, touro, ouro]","[minut, tour, our]"
4,5,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0,"[@, ricktolledo, Sim]","[@, ricktolledo, Sim]","[@, ricktolled, sim]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12350,3044013,"""2022-11-30 11:51:00""",onixcapital,Uma noite de encontros e muito conhecimento on...,NEUTRAL,marcação,1,54,https://www.instagram.com/reel/CllsqI7goSC/,0,0,"[Uma, noite, de, encontros, e, muito, conhecim...","[noite, encontros, conhecimento, onde, cliente...","[noit, encontr, conhec, onde, client, barreira..."
12351,3044212,"""2022-11-30 12:25:00""",amgcapitalinvest,Erros financeiros para eliminar antes dos 30 a...,NEUTRAL,marcação,0,39,https://www.instagram.com/p/Cllw0CmNGkX/,0,0,"[Erros, financeiros, para, eliminar, antes, do...","[Erros, financeiros, eliminar, antes, 30, anos...","[erros, financeir, elimin, antes, 30, anos, .,..."
12352,3044213,"""2022-11-30 12:18:00""",jayipacderota,Estou muito grato por todo o esforço que você ...,POSITIVE,comentário,1,52,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0,"[Estou, muito, grato, por, todo, o, esforço, q...","[grato, todo, esforço, rendeu, investimento, R...","[grat, tod, esforc, rend, invest, r, $, 17.000..."
12353,3044412,"""2022-11-30 12:50:00""",moinho_cultural,"""👉GUADAKAN está chegando!!!! A aula pública e...",NEUTRAL,marcação,0,43,https://www.instagram.com/reel/ClluX-cAoaf/,0,0,"[``, 👉GUADAKAN, está, chegando, !, !, !, !, A,...","[``, 👉GUADAKAN, chegando, !, !, !, !, aula, pú...","[``, 👉guadakan, cheg, !, !, !, !, aul, públic,..."


### 1.2.4 Lemmatização

In [211]:
def lemmatize(tokens):
    if isinstance(tokens, list): # verifica se é um comentário do tipo string
        lemma_list = []
        # Inicializa o lematizador da NLTK
        lemmatizer = WordNetLemmatizer()
        # Lematiza cada token na lista de tokens
        for token in tokens: 
            lemma_list.append(lemmatizer.lemmatize(token))
        return lemma_list
    else:
        return []

In [212]:
df['lemmatized'] = df['no_stopwords'].apply(lemmatize)
df

,id,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink,tokenized,no_stopwords,stemmed,lemmatized
0,1,"""2022-03-04 09:38:00""",v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,marcação,1,52,https://www.instagram.com/p/CarqO6drRux/,0,0,"[Confira, os, resultados, dos, nossos, fundos,...","[Confira, resultados, fundos, mês, fevereiro/2...","[conf, result, fund, mês, fevereiro/22, ., tod...","[Confira, resultados, fundos, mês, fevereiro/2..."
1,2,"""2022-03-04 09:29:00""",winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,marcação,0,43,https://www.instagram.com/p/CarpQUluh7z/,0,0,"[A, Alvarez, &, Marsal, estará, conosco, no, S...","[Alvarez, &, Marsal, estará, conosco, Sportain...","[alvarez, &, marsal, estar, conosc, sportainme...","[Alvarez, &, Marsal, estará, conosco, Sportain..."
2,3,"""2022-03-04 09:20:00""",marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,marcação,0,45,https://www.instagram.com/p/CaroK9Nu8tC/,0,0,"[#, Repost, btgpactual, with, make_repost, ・・・...","[#, Repost, btgpactual, with, make_repost, ・・・...","[#, repost, btgpactual, with, make_repost, ・・・...","[#, Repost, btgpactual, with, make_repost, ・・・..."
3,4,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0,"[Minuto, touro, de, ouro]","[Minuto, touro, ouro]","[minut, tour, our]","[Minuto, touro, ouro]"
4,5,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0,"[@, ricktolledo, Sim]","[@, ricktolledo, Sim]","[@, ricktolled, sim]","[@, ricktolledo, Sim]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12350,3044013,"""2022-11-30 11:51:00""",onixcapital,Uma noite de encontros e muito conhecimento on...,NEUTRAL,marcação,1,54,https://www.instagram.com/reel/CllsqI7goSC/,0,0,"[Uma, noite, de, encontros, e, muito, conhecim...","[noite, encontros, conhecimento, onde, cliente...","[noit, encontr, conhec, onde, client, barreira...","[noite, encontros, conhecimento, onde, cliente..."
12351,3044212,"""2022-11-30 12:25:00""",amgcapitalinvest,Erros financeiros para eliminar antes dos 30 a...,NEUTRAL,marcação,0,39,https://www.instagram.com/p/Cllw0CmNGkX/,0,0,"[Erros, financeiros, para, eliminar, antes, do...","[Erros, financeiros, eliminar, antes, 30, anos...","[erros, financeir, elimin, antes, 30, anos, .,...","[Erros, financeiros, eliminar, ante, 30, ano, ..."
12352,3044213,"""2022-11-30 12:18:00""",jayipacderota,Estou muito grato por todo o esforço que você ...,POSITIVE,comentário,1,52,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0,"[Estou, muito, grato, por, todo, o, esforço, q...","[grato, todo, esforço, rendeu, investimento, R...","[grat, tod, esforc, rend, invest, r, $, 17.000...","[grato, todo, esforço, rendeu, investimento, R..."
12353,3044412,"""2022-11-30 12:50:00""",moinho_cultural,"""👉GUADAKAN está chegando!!!! A aula pública e...",NEUTRAL,marcação,0,43,https://www.instagram.com/reel/ClluX-cAoaf/,0,0,"[``, 👉GUADAKAN, está, chegando, !, !, !, !, A,...","[``, 👉GUADAKAN, chegando, !, !, !, !, aula, pú...","[``, 👉guadakan, cheg, !, !, !, !, aul, públic,...","[``, 👉GUADAKAN, chegando, !, !, !, !, aula, pú..."


### 1.2.4 Bag of Words

In [ ]:
# Função que recebe a coluna ['lemmatized'] e gera um modelo Bag of Words
def bow(lemmatized):
    # Inicializa o CountVectorizer
    vectorizer = CountVectorizer()
    # Cria o modelo Bag of Words
    bow_model = vectorizer.fit_transform(lemmatized)
    # Cria um dataframe com as palavras e suas frequências
    bow_df = pd.DataFrame(bow_model.toarray(), columns=vectorizer.get_feature_names())
    return bow_df

In [213]:
def model_bow(tokens):
    # create the transform
    vectorizer = CountVectorizer()
    # tokenize and build vocab
    vectorizer.fit(tokens)
    # summarize
    print(sorted(vectorizer.vocabulary_))

    # encode document
    vector = vectorizer.transform(tokens)
    # summarize encoded vector
    print(vector.shape)
    print(vector.toarray())

In [214]:
type(df['stemmed'])

pandas.core.series.Series

In [215]:
df['stemmed'].values

array([list(['conf', 'result', 'fund', 'mês', 'fevereiro/22', '.', 'tod', 'inform', ',', 'document', 'materi', 'técnic', 'fund', 'pod', 'acess', 'links', 'abaix', ':', '>', 'v8', 'cash', 'fi', 'rend', 'fix', 'cp', ':', 'https', ':', '//lnkd.in/dusc2jmn', '>', 'v8', 'vanquish', 'term', 'fi', 'rend', 'fix', ':', 'https', ':', '//lnkd.in/d3pngjuu', '>', 'v8', 'veyron', 'smart', 'bet', 'fi', 'açõ', ':', 'https', ':', '//lnkd.in/djnjw4pp', '>', 'v8', 'elva', 'deb', '.', 'incentiv', 'fic', 'fim', 'cp', ':', 'https', ':', '//lnkd.in/dap-37vv', '>', 'v8', 'challeng', 'fim', 'cp', ':', 'https', ':', '//lnkd.in/d5bnhzqj', '>', 'v8', 'anf', 'luxbridg', 'global', 'real', 'estat', 'fim', 'ie', ':', 'https', ':', '//lnkd.in/dxzcatpp', 'onde', 'encontr', 'fund', ':', 'modalm', 'oramainvest', 'warrenbrasil', 'terrainvestimentosoficial', 'ativainvest', 'novafuturainvest', 'nubank', 'nectoninvest', 'miraeassetcorretor', 'toroinvest', 'rbinvest', 'guideinvest', 'btgpactual', 'xpinvest', 'simpaulinvest', 

In [216]:
type(df['stemmed'].values)

numpy.ndarray

In [217]:
v = CountVectorizer()
vocab = v.fit_transform((df['stemmed'].values)[0])
vocab

<123x73 sparse matrix of type '<class 'numpy.int64'>'
	with 105 stored elements in Compressed Sparse Row format>

In [218]:
df['texto'][0]

'Confira os resultados dos nossos fundos no mês de fevereiro/22. Todas as informações, documentações e materiais técnicos dos fundos podem ser acessados pelos links abaixo:  > V8 Cash FI Renda Fixa CP: https://lnkd.in/dusC2Jmn > V8 Vanquish Termo FI Renda Fixa: https://lnkd.in/d3PnGJUu > V8 Veyron Smart Beta FI Ações: https://lnkd.in/dJnJW4PP > V8 Elva Deb. Incentivadas FIC FIM CP: https://lnkd.in/dap-37vV > V8 Challenger FIM CP: https://lnkd.in/d5bNHZQJ > V8 ANF Luxbridge Global Real Estate FIM IE: https://lnkd.in/dXZcaTpp  Onde você encontra nossos fundos: modalmais oramainvestimentos warrenbrasil terrainvestimentosoficial ativainvestimentos novafuturainvestimentos nubank nectoninvestimentos miraeassetcorretora toroinvestimentos rbinvestimentos guideinvestimentos btgpactual xpinvestimentos simpaulinvestimentos   #v8capital #investir #investimentos #mercadofinanceiro #ibovespa #cdi #rendafixa #multimercado #ações'

In [219]:
# final = df['stemmed']

# model_bow(final)

In [220]:
# result